In [ ]:
import re
import h5py, os
import numpy as np

# test_cfg='0006_r2'

inpath='data_post/'

cfgs = [cfg for cfg in os.listdir(inpath)]
cfgs.sort()

cfgSrcDic={}
for cfg in cfgs:
    cfgSrcDic[cfg]={}
    for file in os.listdir(inpath+cfg):
        if not file.startswith('N.h5'):
            continue
        with h5py.File(inpath+cfg+'/'+file) as f:
            cfgSrcDic[cfg][file]=list(f['data'].keys())
    # break

In [ ]:
for cfg in cfgs:
    files=list(cfgSrcDic[cfg].keys())
    files.sort()
    for file in files:
        if len(cfgSrcDic[cfg][file])!=650:
            print(cfg+': '+file,len(cfgSrcDic[cfg][file]),end='\t')
    # print()

In [ ]:
import tarfile
inpath='/p/project/pines/li47/code/projectData/discNJN/others/thrp-twop.tar'
tar = tarfile.open(inpath)
for tarinfo in tar:
    if 'cC80' not in tarinfo.name:
        continue
    handle = tar.extractfile(tarinfo)
    if 'twop' in tarinfo.name:
        with h5py.File(handle) as f:
            t1=list(f.keys())
    else:
        with h5py.File(handle) as f:
            t2=list(f['dt10_twop'].keys())
            
def getcfgs(file,Nsrc):
    cfgs0=[]
    for cfg in cfgs:
        if len(cfgSrcDic[cfg][file])==Nsrc:
            cfgs0.append(cfg)
    cfgs0.sort()
    return cfgs0
            
def compare(cfgs1,cfgs2):
    print('1:',len(cfgs1),'; 2:',len(cfgs2))
    cfgs_full=set(cfgs1)|set(cfgs2)
    cfgs_com=set(cfgs1)&set(cfgs2)
    print('full:',len(cfgs_full),'; com:',len(cfgs_com))
    print('in 1 not in 2: ',cfgs_full-set(cfgs2))
    print('in 2 not in 1: ',cfgs_full-set(cfgs1))
    print()

compare(t1,t2)
compare(t1,getcfgs('N.h5_twop_nucl_srcs650',650))

cfgs_final=getcfgs('N.h5_twop_nucl_srcs650',650)

cfgs_final=list(cfgs_final)
cfgs_final.sort()
with open('cfgs_final','w') as f:
    for cfg in cfgs_final:
        f.write(cfg+'\n')

In [ ]:
import re
import h5py, os
import numpy as np

test_cfg='0344_r0'

inpath='data_post/'

cfgs = [cfg for cfg in os.listdir(inpath)]
cfgs.sort()

cfgSrcDic={}
for cfg in [test_cfg]:
    cfgSrcDic[cfg]={}
    for file in os.listdir(inpath+cfg):
        if not file.startswith('N.h5'):
            continue
        with h5py.File(inpath+cfg+'/'+file) as f:
            cfgSrcDic[cfg][file]=list(f['data'].keys())
    # break

path='/p/arch/hch02/hch02b/cD211.54.96/'
srcs={}
for dir in ['twop_threep','twop_threep_2','twop_threep_3']:
    t_path=path+dir+'/twop_'+test_cfg+'.h5'
    with h5py.File(t_path) as f:
        srcs[dir]=[]
        for src in f['baryons/nucl_nucl/twop_baryon_1'].keys():
            (sx,sy,sz,st)=re.search('sx([0-9]*)sy([0-9]*)sz([0-9]*)st([0-9]*)',src).groups()
            (sx,sy,sz,st)=(int(sx),int(sy),int(sz),int(st))
            src_new='sx'+str(sx)+'sy'+str(sy)+'sz'+str(sz)+'st'+str(st)
            srcs[dir].append(src_new)

srcs['nucl_all_source']=cfgSrcDic[test_cfg]['N.h5_nucl_all_source']
srcs['twop_129srcs']=cfgSrcDic[test_cfg]['N.h5_twop_129srcs']    

def run(a,b):
    print(a,len(srcs[a]),b,len(srcs[b]))
    print(np.sum([1 if src in srcs[b] else 0 for src in srcs[a]]))

ss=(s_all,s_129,s_1,s_2,s_3)=('nucl_all_source','twop_129srcs','twop_threep','twop_threep_2','twop_threep_3')
for s in ss:
    print(s,len(srcs[s]))
print()
run(s_129,s_all)
run(s_1,s_all)
run(s_2,s_all)
run(s_3,s_all)
run(s_3,s_2)
print()
run(s_1,s_3)
run(s_2,s_3)
run(s_all,s_3)